<img src='img/img_1.png'>
<img src='img/img_2.png'>
<img src='img/img_3.png'>

In [1]:
import pandas as pd

data = pd.read_csv('turnover.csv')

In [2]:
data.head()

,satisfaction,evaluation,number_of_projects,average_montly_hours,time_spend_company,work_accident,churn,promotion,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
satisfaction            14999 non-null float64
evaluation              14999 non-null float64
number_of_projects      14999 non-null int64
average_montly_hours    14999 non-null int64
time_spend_company      14999 non-null int64
work_accident           14999 non-null int64
churn                   14999 non-null int64
promotion               14999 non-null int64
department              14999 non-null object
salary                  14999 non-null object
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [4]:
data.salary.unique()

array(['low', 'medium', 'high'], dtype=object)

In [5]:
data.department.unique()

array(['sales', 'accounting', 'hr', 'technical', 'support', 'management',
       'IT', 'product_mng', 'marketing', 'RandD'], dtype=object)

#Transforming Categorical Variables
<img src='img/img_4.png'>

In [6]:
data.salary = data.salary.astype('category')

In [7]:
data.salary = data.salary.cat.reorder_categories([
    'low', 'medium', 'high'
])

In [8]:
data.salary = data.salary.cat.codes

In [9]:
departments = pd.get_dummies(data.department)
print(departments.head())

   IT  RandD  accounting  hr  management  marketing  product_mng  sales  \
0   0      0           0   0           0          0            0      1   
1   0      0           0   0           0          0            0      1   
2   0      0           0   0           0          0            0      1   
3   0      0           0   0           0          0            0      1   
4   0      0           0   0           0          0            0      1   

   support  technical  
0        0          0  
1        0          0  
2        0          0  
3        0          0  
4        0          0  


In [10]:
departments = departments.drop('accounting', axis=1)
data = data.drop('department', axis=1)
data = data.join(departments)

In [11]:
n_employees = len(data)
print(data.churn.value_counts())
print('**************************')
print(data.churn.value_counts() / n_employees* 100)

0    11428
1     3571
Name: churn, dtype: int64
**************************
0    76.191746
1    23.808254
Name: churn, dtype: float64


In [12]:
target = data.churn
features = data.drop('churn', axis=1)
print(target.shape)
print(features.shape)

(14999,)
(14999, 17)


In [13]:
from sklearn.model_selection import train_test_split

trainY, testY, trainX, testX = train_test_split(target,
               features, test_size=0.25, random_state=42)

print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(11249, 17) (11249,)
(3750, 17) (3750,)


In [14]:
stayed = 37
left = 1138

total =stayed + left
gini = 2*(stayed / total)*(left /  total)
print(gini)

0.060995563603440474


In [15]:
gini_A = 0.65
gini_B = 0.15

if gini_A < gini_B:
    print('split by A')
else: 
    print('split by B')

split by B


In [16]:
from sklearn.tree import DecisionTreeClassifier

In [17]:
model = DecisionTreeClassifier(random_state=42)

In [18]:
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [19]:
model.fit(trainX, trainY)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [20]:
model.score(testX, testY) * 100

97.22666666666666

In [21]:
from sklearn.tree import export_graphviz

export_graphviz(model, 'tree.dot')

In [23]:
model_depth_5 = DecisionTreeClassifier(max_depth=5, random_state=42)
model_depth_5.fit(trainX, trainY)
print(model_depth_5.score(trainX, trainY)*100)
print(model_depth_5.score(testX, testY)*100)

97.71535247577563
97.06666666666666


In [24]:
# initialize the decision tree classifier while limiting the 
# sample  size in leaves to 100
model_sample_100 = DecisionTreeClassifier(min_samples_leaf=100, random_state=42)
model_sample_100.fit(trainX, trainY)
print(model_sample_100.score(trainX, trainY)*100)
print(model_sample_100.score(testX, testY)*100)


96.57747355320473
96.13333333333334


In [29]:
from sklearn.metrics import precision_score

prediction = model_sample_100.predict(testX)
print(precision_score(testY, prediction))

0.9302059496567505


In [31]:
from sklearn.metrics import recall_score

print(recall_score(testY, prediction))

0.9063545150501672


In [32]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(testY, prediction))

0.9424867562983048


In [33]:
model_depth_5_b = DecisionTreeClassifier(max_depth=5, class_weight='balanced',
                            random_state=42)
model_depth_5_b.fit(trainX, trainY)

print(model_depth_5_b.score(testX, testY)*100)

93.70666666666668


In [34]:
model_depth_7_b = DecisionTreeClassifier(max_depth=7,class_weight="balanced",random_state=42)
# Fit it to the training component
model_depth_7_b.fit(trainX,trainY)
# Make prediction using test component
prediction_b = model_depth_7_b.predict(testX)
# Print the recall score for the balanced model
print(recall_score(testY,prediction_b))
# Print the ROC/AUC score for the balanced model
print(roc_auc_score(testY,prediction_b))

0.9319955406911928
0.959863876199084


In [35]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(model_depth_5_b, trainX, trainY, cv=10))

[0.93694494 0.94316163 0.93605684 0.94493783 0.93955556 0.93772242
 0.94395018 0.93594306 0.94483986 0.94928826]


In [36]:
depth = [i for i in range(5, 21, 1)]

samples = [i for i in range(50, 500, 50)]

parameters = dict(max_depth=depth, min_samples_leaf=samples)

In [37]:
from sklearn.model_selection import GridSearchCV

param_search = GridSearchCV(model_depth_7_b, parameters)

param_search.fit(trainX, trainY)
print(param_search.best_params_)

/home/apolanco/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'max_depth': 6, 'min_samples_leaf': 50}


In [40]:
model_best = DecisionTreeClassifier(max_depth=6, min_samples_leaf=50, 
                                   random_state=42)
model_best.fit(trainX, trainY)

feature_importances = model_best.feature_importances_

In [49]:
features_list = ['satisfaction',
 'evaluation',
 'number_of_projects',
 'average_montly_hours',
 'time_spend_company',
 'work_accident',
 'promotion',
 'salary',
 'IT',
 'RandD',
 'hr',
 'management',
 'marketing',
 'product_mng',
 'sales',
 'support',
 'technical']

In [53]:
relative_importances = pd.DataFrame(index=features_list, data=feature_importances,
                               columns=['importance'])

print(relative_importances.sort_values(by='importance', ascending=False))

                      importance
satisfaction            0.552654
time_spend_company      0.155925
evaluation              0.143357
number_of_projects      0.092223
average_montly_hours    0.054417
technical               0.000902
hr                      0.000293
salary                  0.000230
promotion               0.000000
work_accident           0.000000
RandD                   0.000000
management              0.000000
marketing               0.000000
product_mng             0.000000
sales                   0.000000
support                 0.000000
IT                      0.000000


In [62]:
selected_features = relative_importances[relative_importances.importance>0.01]

selected_list = selected_features.index

features_train_selected = trainX[selected_list]
features_test_selected = testX[selected_list]

In [64]:
model_best = DecisionTreeClassifier(max_depth=8, min_samples_leaf=150, class_weight='balanced', random_state=42)

# Fit the model using only selected features from training set: done
print(features_train_selected.shape, trainY.shape)


(11249, 5) (11249,)


In [70]:
model_best.fit(features_train_selected, trainY)

# Make prediction based on selected list of features from test set
prediction_best = model_best.predict(features_test_selected)

# Print the general accuracy of the model_best
print(model_best.score(features_test_selected, testY) * 100)

# Print the recall score of the model predictions
print(recall_score(testY, prediction_best) * 100)

# Print the ROC/AUC score of the model predictions
print(roc_auc_score(testY, prediction_best) * 100)

95.28
91.75027870680044
94.07002193314084
